In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F

from src.models.distance_modeling import SurveyorDataSet, Surveyor, thesis_kl_div_add_mse_loss, RetrieveSystem

import copy
from src.models.DenseNetwork import loss

torch.manual_seed(0)

In [3]:
device = torch.device('cuda:0')

In [4]:
best_model = Surveyor()
best_model.load_state_dict(torch.load('../saves/surveyor.on.full.0.7675')['best_model'])
best_model.eval()

val_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/dev.csv')
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1000, pin_memory=True)

gold = loss.nearest_neighbors(val_dataset.data.clone(), top_k=1, device='cuda')

In [5]:
best_model = best_model.to(device)

In [6]:
def extract_embeddings(data_loader, model):
    model.eval()
    embedding = list()
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            x = batch[0].to(device)
            out = model.encode_batch(x)
            embedding.append(out.cpu())
    return torch.cat(embedding, dim=0)

val_nn_loader = DataLoader(TensorDataset(val_dataset.data.clone()), batch_size=1000, pin_memory=True)
val_x_embedded = extract_embeddings(val_nn_loader, best_model)

In [7]:
val_x_embedded

tensor([[-1.1144,  1.0543, -1.9614,  ..., -1.3660, -1.4025, -0.3738],
        [-1.1302,  1.0884, -1.9891,  ..., -1.3714, -1.3870, -0.3723],
        [-0.9590,  1.4835, -1.2173,  ..., -1.5760, -1.4293,  0.3513],
        ...,
        [-0.9589,  1.4845, -1.2212,  ..., -1.5773, -1.4346,  0.3427],
        [ 2.1078, -0.0564, -0.5392,  ..., -0.7730,  0.3463,  1.2952],
        [ 2.1118, -0.0587, -0.5391,  ..., -0.7726,  0.3467,  1.2980]])

In [8]:
retriever = RetrieveSystem(best_model)
block_list = torch.arange(val_x_embedded.shape[0])
cls_pred_nn_top, p_distances_nn_top = retriever.retrieve_corpus(val_x_embedded, block_list, val_x_embedded)


In [9]:
print(cls_pred_nn_top.shape)
print(p_distances_nn_top.shape)
print(gold[2].shape)

torch.Size([1000, 20])
torch.Size([1000, 20])
torch.Size([1000, 1])


In [10]:
# recall @1
float((cls_pred_nn_top[:, 0] == gold[1].view(-1)).sum()) / len(gold)

0.0

In [11]:
# recall@5
float((cls_pred_nn_top[:, :5] == gold[1]).sum()) / len(gold[1])

0.006

In [12]:
# recall@10
float((cls_pred_nn_top[:, :10] == gold[1]).sum()) / len(gold[1])

0.01

In [13]:
# recall@20
float((cls_pred_nn_top[:, :20] == gold[1]).sum()) / len(gold[1])

0.025